In [ ]:
import numpy as np
import torch
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)
batch_size = 1
seq_len = 5
INPUT_DIM = 10
HIDDEN_DIM = 20
SUM_DIM = INPUT_DIM + HIDDEN_DIM
x = torch.randn(batch_size, seq_len, INPUT_DIM)
h_0 = torch.zeros((batch_size, HIDDEN_DIM))
c_0 = torch.zeros((batch_size, HIDDEN_DIM))

'''
initialize the weights of each layer
'''
Wf = torch.randn(HIDDEN_DIM,SUM_DIM)
Wi = torch.randn(HIDDEN_DIM,SUM_DIM)
Wo = torch.randn(HIDDEN_DIM,SUM_DIM)
Wc = torch.randn(HIDDEN_DIM,SUM_DIM)

bf = torch.randn(HIDDEN_DIM, batch_size)
bi = torch.randn(HIDDEN_DIM, batch_size)
bo = torch.randn(HIDDEN_DIM, batch_size)
bc = torch.randn(HIDDEN_DIM, batch_size)


x_0 = x[:, 0, :].reshape(batch_size, -1)
combined = torch.concat([h_0, x_0],dim=1)

f_t = sigmoid((Wf @ combined.T) + bf)
i_t = sigmoid((Wi @ combined.T) + bi)
o_t = sigmoid((Wo @ combined.T) + bo)
c_hat_t = tanh((Wc @ combined.T) + bc)

f_t = f_t.permute(1,0)
i_t = i_t.permute(1,0)
o_t = o_t.permute(1,0)
c_hat_t = c_hat_t.permute(1,0)

c_t = f_t * c_0 + i_t * c_hat_t # next cell state
h_t = o_t * tanh(c_t) # next hidden state

In [ ]:
import numpy as np

class Activations(object):
  def sigmoid(x):
      return 1 / (1 + np.exp(-x))

  def tanh(x):
      return np.tanh(x)

class LSTM:
    def __init__(self, input_dim, hidden_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        # Initialize weights
        self.Wf = torch.randn(self.hidden_dim, self.hidden_dim + self.input_dim)
        self.Wi = torch.randn(self.hidden_dim, self.hidden_dim + self.input_dim)
        self.Wo = torch.randn(self.hidden_dim, self.hidden_dim + self.input_dim)
        self.Wc = torch.randn(self.hidden_dim, self.hidden_dim + self.input_dim)

        # Initialize bias
        self.bf = torch.zeros((self.hidden_dim, 1))
        self.bi = torch.zeros((self.hidden_dim, 1))
        self.bo = torch.zeros((self.hidden_dim, 1))
        self.bc = torch.zeros((self.hidden_dim, 1))

    def forward(self, x, h_t, c_t):
        batch_size, sequence_len, _ = x.shape
        h_ts = []
        for t in range(sequence_len):
            x_t = x[:, t, :].reshape(batch_size, -1)
            combined = torch.concat([h_t, x_t],dim=1)

            f_t = Activations.sigmoid((self.Wf @ combined.T + self.bf))
            i_t = Activations.sigmoid((self.Wi @ combined.T + self.bi))
            o_t = Activations.sigmoid((self.Wo @ combined.T + self.bo))
            c_hat_t = Activations.tanh((self.Wc @ combined.T + self.bc))

            f_t = f_t.permute(1,0)
            i_t = i_t.permute(1,0)
            o_t = o_t.permute(1,0)
            c_hat_t = c_hat_t.permute(1,0)

            c_t = f_t * c_t + i_t * c_hat_t
            h_t = o_t * Activations.tanh(c_t)
            h_ts.append(h_t)

        return h_t, c_t, h_ts  # Return the final state

    def initHidden(self, batch_size=1):
        return torch.zeros((batch_size,self.hidden_dim))
    def initCellState(self, batch_size=1):
        return torch.zeros((batch_size,self.hidden_dim))

# Define parameters
INPUT_DIM = 10
HIDDEN_DIM = 20
seq_len = 5
batch_size = 64
# Create LSTM instance
lstm = LSTM(INPUT_DIM, HIDDEN_DIM)

# Create dummy input
x = torch.randn(batch_size, seq_len, INPUT_DIM)  # (batch_size, sequence_length, input_dim)
h0 = lstm.initHidden(batch_size)
c0 = lstm.initCellState(batch_size)
# combined = np.column_stack((h0, x[:,0,:]))
# combined.shape

# # Forward propagate
h, c, out = lstm.forward(x, h0, c0)
# print("Last hidden state:", h)


In [ ]:
h.shape, c.shape, out[-1].shape

(torch.Size([64, 20]), torch.Size([64, 20]), torch.Size([64, 20]))

In [ ]:
len(out)

5